# Standaard block voor ophalen weersgegevens
Onderstaande code is een standaard block voor het opvragen van historische weersgegevens. De API die is gebruikt, is 'Visual Crossing Weather' ontwikkeld door Visual Crossing Corporation.

Meer info over de API: https://rapidapi.com/visual-crossing-corporation-visual-crossing-corporation-default/api/visual-crossing-weather/endpoints


## Aan te passen variabele
De onderstaande variabele (Datum en Locatie) zijn redelijk vrij aan te passen. Het is darentegen wel belangrijk dat de huidige datumnotatie gehanteerd blijft (dd-mm-yyyy). De notatie van de locatie is echter niet hoofdlettergevoelig.

In [2]:
Datum = "01-01-2019"
Locatie = "Rotterdam"

## Standaard aanpak voor het verwerken van de reactie
Onderstaand blok is de standaard aanpak voor het verwerken van de reactie die wordt ontvangen van de API. De weerdata van de desbetreffende dag op de desbetreffende locatie wordt gegeven in de vorm van een doctionary. 

### Importeren van de libraries

In [1]:
from datetime import datetime

import pandas as pd
import requests
from pandas.io.json import json_normalize

### Omvormen van de datum naar het correcte format voor in de querystring voor de API
Om het aanpassen van de datum zo gemakkelijk mogelijk te houden, wordt de datum gevraagd in de verkorte Europese notatie (dd-mm-jjjj) en vervolgens door onderstannde cell omgevormd in de notatie die de API begrijpt (yyyy-mm-dd voor in yyyy-mm-ddTuu:mm:ss+[tijdzone]).

In [3]:
dateObject = datetime.strptime(Datum, "%d-%m-%Y").strftime('%Y-%m-%d')

### Vastleggen van de url
De url is het adres van de API. Hier wordt de request naar gestuurd.

In [4]:
url = 'https://visual-crossing-weather.p.rapidapi.com/history'

### Definiëren van de headers
Headers moeten gespecificeerd worden. De headers geven meta-data van de request en response van de API en authoriseren de aanvraag en response.

In [6]:
headers = {'x-rapidapi-host': "visual-crossing-weather.p.rapidapi.com",
           'x-rapidapi-key': "01760723f2msh9ad74e49c3fa409p14af2fjsn481f178a56b5"}

### Definiëren van de querystring
De querystring maakt de aanvraag concreet en definieerd de variabelen van de aanvraag. Om het aanpassen van de locatie zo eenvoudig mogelijk te maken, wordt in de querystring de locatie in de api-specifieke notatie geplakt. In deze toepassing worden de weersgegevens alleen opgevraagd van Nederlandse steden, waardoor 'NL' niet gespecificeerd hoeft te worden door de gebruiker.

In [5]:
querystring = {"dayStartTime": "8:00:00",
               "contentType": "json",
               "dayEndTime": "17:00:00",
               "shortColumnNames": "false",
               "startDateTime": f'{dateObject}T00:00:00',
               "aggregateHours": "24",
               "location": f'{Locatie.capitalize()}%2C NL',  # NL hoeft niet variabel te zijn, geen projecten buiten NL
               "endDateTime": f'{dateObject}T00:00:00',
               "unitGroup": "metric"}

### API bevragen

In [7]:
response = requests.request("GET", url, headers=headers, params=querystring)

### De response omvormen tot een dataframe
De response wordt ontvangen in json format, maar de inhoud is nog enkel aan te spreken als string (tekst). Deze moet omgevormd worden om het gemakkelijk aanspreekbaar te maken.

In [8]:
r = response.json()
df = json_normalize(r)

C:\Users\NBais\AppData\Roaming\Python\Python37\site-packages\ipykernel_launcher.py:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  


### Isoleren van de pd.Series met de waarden uit het dataframe
Het dataframe bevat één rij en groot aantal kolommen, waarvan één kolom alle gewenste waarden bevat in de vorm van een dict(). Dit moet geïsoleerd worden om verder handelingen te vereenvoudigen.

In [9]:
df2 = df[f'locations.{Locatie.capitalize()}%2C NL.values']

### Isoleren van de dict() in de pd.Series

In [10]:
_d = df2.values
d = _d[0][0]

### Definiëren van de gewenste waarden
De waarde die geëxtraheerd moeten worden, moeten vooraf gedefinieerd worden om ze vervolgens toe te kunnen passen in een f-string. Dit als 'work-around'.

In [11]:
temp = d['temp']  # Gemiddelde temperatuur
mint = d['mint']  # Minimale temperatuur
maxt = d['maxt']  # Maximale temperatuur
humidity = d['humidity']  # Luchtvochtigheid
condition = d['conditions']  # Beschrijving van de weerscondities

### Definiëren van een dataframe met enkel de gewenste data

In [12]:
weerdata = pd.DataFrame([{temp, mint, maxt, humidity, condition}],
                        columns=['temp', 'min temp', 'max temp', 'humidity', 'condition'])

### Het presenteren van het resultaat
Deze stap print het dataframe om te laten zien dat het gewenste resultaat behaald is.

In [13]:
print(weerdata)

   temp  min temp  max temp  humidity               condition
0   5.3       7.1       9.0      75.3  Rain, Partially cloudy
